In [1]:
from __future__ import division
import numpy as np
import pandas as pd
import scipy.stats as st
import itertools
import math
from collections import Counter, defaultdict
%load_ext autoreload
%autoreload 2

In [26]:
import nltk
import string
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer

In [3]:
import cPickle as pickle

In [110]:
with open('./flavor_compund_map.pkl', 'r') as f:
    raw_flav_ing = pickle.load(f)

In [7]:
ingredient_bow = ' '.join(raw_flav_ing.values())

## Seperating individual ingredients

I need to-
take out common connective words
lower
remove punct?

In [27]:
ing_words = word_tokenize(ingredient_bow)

## ing_words

In [32]:
with open('./data/recipes.pkl', 'r') as f:
    recipe_raw = pickle.load(f)

In [41]:
recipe_comb = {}
for f in recipe_raw:
    recipe_comb.update(f)

In [44]:
raw_flav_ing

In [57]:
cocktail_ing_lists = recipe_comb.values()
cocktail_ing_bow = [ing[0].lower() for ing in cocktail_ing_lists]
len(set(cocktail_ing_bow))

In [58]:
ingredient_bow[:1000]

In [75]:
wcs = Counter(ingredient_bow.split(','))
wc_common = wcs.most_common()

In [82]:
ingredients_list_from_fen = [ing[0].strip().replace(',','') for ing in wc_common if ing[1]>1]
fen_set = set(ingredients_list_from_fen)

In [83]:
len(fen_set)

In [87]:
fen_set

In [86]:
fen_set.intersection(set(cocktail_ing_bow))

In [117]:
def replace_with_ing_lists(ing_st):
    new_list = []
    potential_ing = ing_st.split(',')
    for ing in potential_ing:
        if ing.strip().replace(',','') in fen_set:
            new_list.append(ing.strip().replace(',',''))
    return new_list

In [121]:
with open('./flavor_compund_map.pkl', 'r') as f:
    raw_flav_ing = pickle.load(f)

In [119]:
for comp, ing_st in raw_flav_ing.iteritems():
    raw_flav_ing[comp] = replace_with_ing_lists(ing_st)

In [120]:
raw_flav_ing

Splitting on the commas, while easy, misses ingredients that appear as below

In [122]:
raw_flav_ing['2,6-DIMETHYL-4-HEPTANONE']